In [2]:
import torch.optim as optim
from torch import nn
from gloss_model import GlossModel
from torch.utils.tensorboard import SummaryWriter


In [ ]:
# provide input and class size
model = GlossModel()
optim = optim.SGD(model.parameters(), lr=0.1)
loss_fn= nn.CrossEntropyloss()

In [ ]:
# split x , y for training and set epoch

In [ ]:
# initialize sumamry writer
writer=SummaryWriter()

In [ ]:
# Start model training
model.train()

for i in range(epoch):
    optim.zero_grad()
    out = model(x_train)
    loss = loss_fn(out, y_train.argmax())
    loss.backward()
    optim.step()
    writer.add_scalar("Loss/epoch", loss.item(), i)

writer.close()
